In [1]:
import time
from datetime import datetime
sc = SparkContext.getOrCreate()
from pyspark.sql.functions import desc
#traces
import pandas as pd
import numpy as np
import re
import sys
import math
#for ordered dict for traces implementation
from collections import OrderedDict
#from pyspark.sql.types import VectorUDT
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType

from pyspark.sql.functions import udf

from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

from pyspark.sql import Row

#kmeans
#from pyspark.mllib.clustering import KMeans, KMeansModel
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
#from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

from pyspark.ml.feature import StandardScaler

from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [2]:
%load_ext autotime
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [3]:
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def fix_df(df):
    return df.rdd.toDF(df.schema.names)


def change_column_names(df, old_names, new_names):
    pass
    return df;

time: 6.62 ms


In [4]:
#df_from_csv('f_cluster.csv')

time: 835 µs


In [5]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    uf = udf(lambda x: 'new_value', StringType())
    s_udf = udf(lambda value: value*self.mmModel.getWithStd() + self.mmModel.getWithMean())
    
    
    def __init__(self, inputCol, outputCol, s_min = 0, s_max = 0):
        #self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.mmModel = StandardScaler(inputCol=inputCol, outputCol=outputCol)
        self.mmModel.setWithMean(True)
        #self.mmModel.setMin(s_min)
        #self.mmModel.setMax(s_max)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
    
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        #self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        #self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        #udf = UserDefinedFunction(lambda x: 'new_value', StringType())
        print 'deno'
        return 5
        #return value*self.mmModel.getWithStd() + self.mmModel.getWithMean()
        #return 0
        #v = (value-self.getMin())*\
        #    (self.originalMax - self.originalMin)*\
        #    (self.getMax()-self.getMin()) + self.originalMin
        #if v or v == 0:
        #    return v
        #else:
        #    return -999
        
    def denormalize_df(self, df):
        col = self.mmModel.getInputCol()
        print col
        
        
    def normalize(self, value):
        pass

time: 72.5 ms


In [6]:
sc

time: 3.1 ms


In [63]:
#calculate sse
def compute_sse(a,b):
    total = 0
    if len(a) != len(b):
        print 'bad input'
        return 99999
    else:
        for i in range(len(a)):
            total += math.pow(float(a[i])-float(b[i]),2)
    return math.sqrt(total)

def change_type_col_double(df, col):
    df = df.withColumn(col, df[col].cast("double"))
    return df.na.fill(0)

#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

def normalize_vectorize_df(df, list_cols, scale_dict={}):
    n_df = vectorize_columns(df, list_cols)
    n_df, scalerModels = normalize_df(n_df, list_cols, scale_dict)
    return n_df, scalerModels
#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols, debug=False):
    a = datetime.now()
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    b = datetime.now()
    print 'vectorize done in:', b-a
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df_old(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    #print 'masa1', df.select(['geo_distance','diff_altitude','workoutid']).describe().show(10,False)

    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #take out values based on std
        #print df.show()
        df = df.withColumn(col_name, unvector(df[col_name]))
        df = df.filter(df[col_name]<3)
        #print df.count() , 'a'
        #df = df.filter(df[col_name]>0.1)
        #print df.count(), 'b'
        #print 'pd'
        # f_cluster_df.select(route_cols).filter(f_cluster_df['diff_altitude']<3)\
    #%%!.filter(f_cluster_df['geo_distance']<3).toPandas().describe()
        #unv_df = df.select(['diff_altitude', 'geo_distance'])
        #unv_df = unv_df.withColumn('diff_altitude', unvector(df['diff_altitude']))
        #unv_df = unv_df.withColumn('geo_distance', unvector(df['geo_distance']))
        
        
        #print unv_df.select(list_cols).toPandas().describe()
        #r_dict[col_name] = scaler
        #r_dict[index] = scaler
        r_dict[col_name] = scalerModel
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


# MinMaxScaler doesn't have originalMin(only supported in 2.0). made one above cell with wrapper class
# #generate normalized dataframe
# #keep the original column names
# #returns:
# #   (df, {col_name:scalerModel, col_name:scalerModel,....})
# def normalize_df(df, list_cols):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     for col_name in list_cols:
#         scaler = MinMaxScaler(inputCol=col_name, outputCol=tmp_col_name)
#         scalerModel = scaler.fit(df)
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         r_dict[col_name] = [scaler, scalerModel]
#     return df, r_dict

#input: (MLLIB KMEANS)
#   cols_to_cluster: list of column names to cluster
# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
#     vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
#         outputCol='features')
#     df_to_cluster = vecAssembler.transform(df.na.fill(0))
#     df_to_cluster = df_to_cluster.select('features')
#     df_to_cluster = df_to_cluster.rdd\
#         .map(lambda row : Vectors.dense([item for item in row]))
#     clusters = KMeans.train(df_to_cluster, num_clusters,\
#                            maxIterations = 10,\
#                         initializationMode="random")
    
#     return clusters


# #create feature column from list of cols
# def create_feature_column(df, cols_to_cluster):
#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')

#     normalized_df = df
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     #normed_df = df_to_cluster.select(normalized_names)
#     return df_to_cluster   
    
def create_feature_column(df, cols_to_cluster):
    vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
        outputCol='features')

    df_to_cluster = vecAssembler.transform(df.na.fill(0))
    
    #normed_df = df_to_cluster.select(normalized_names)
    return df_to_cluster   
    
    
    
#ML KMEANS
#returns KMeanModel, df_to_cluster
def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=True, bisectingKMean = True,\
                      debug = False):
    a = datetime.now()

#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
#    df_to_cluster = df_to_cluster.select(['workoutid','features'])
    #df_to_cluster = df_to_cluster.rdd\
     #   .map(lambda row : Vectors.dense([item for item in row]))
    #print df.count(), 'masa3'
    df_to_cluster = create_feature_column(df, cols_to_cluster)
    #print df_to_cluster.count(), 'masa4'
        
        
    if not bisectingKMean:
        if not init_var:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
                           featuresCol=cols_to_cluster)
        else:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
    else:
        kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
        #kmeans = BisectingKMeans().setK(num_clusters).setSeed(1).setMinDivisibleClusterSize(10)

    #print df_to_cluster.show()
    model = kmeans.fit(df_to_cluster)
    
    b = datetime.now()
    if debug:
        print 'clustering in ', b-a
    return model, df_to_cluster



# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=False):
#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df.copy()
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
# #    df_to_cluster = df_to_cluster.select(['workoutid','features'])
#     #df_to_cluster = df_to_cluster.rdd\
#      #   .map(lambda row : Vectors.dense([item for item in row]))
        
#     print 'clustering times'
#     a = datetime.now()
#     if not init_var:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
#                        featuresCol=cols_to_cluster)
#     else:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
#     b = datetime.now()
#     print b - a
    
#     model = kmeans.fit(df_to_cluster)
    
#     c = datetime.now()
#     print c - b
#     return model, df_to_cluster

    
#returns clusters. show cluster centerse by clusters.clusterCenters.
#output:
#   [0]: KMeanModel
#   [1]: DF that will be denormalized
#   [2]: ScalerModels
#   [3]: error
#   [4]: DF with features
def cluster_df(df, cols_to_normalize, num_clusters = 2, return_sse = False, cols_to_cluster = [], init_var=True,\
              scale_dict = {}, bisectingKMean = True, debug = False):
    print 'cluster start'
    if not cols_to_cluster:
        cols_to_cluster = cols_to_normalize
    df = change_type_cols_double(df, cols_to_normalize)
    #print 'masa2', df.select(['geo_distance','diff_altitude','workoutid']).describe().show(10,False)
    df = vectorize_columns(df, cols_to_normalize, debug)
    #print 'masa3', df.select(['geo_distance','diff_altitude','workoutid']).describe().show(10,False)
    df, scalerModels = normalize_df(df, cols_to_cluster, scale_dict, debug)
    #print 'masa4', df.select(['geo_distance','diff_altitude','workoutid']).describe().show(10,False)
    
    ##*******************
    #df is object to cluster. if creating kmean models off of subset of cluster, 
    #make subset here
    
    ##*******************
    
    r_obj = cluster_summary_df(df, cols_to_cluster, num_clusters, init_var, bisectingKMean, \
                              debug)
    sse = 'none'
    if return_sse:
        #calculate SSE for this kmean model
        kmodel = r_obj[0]
        df_to_cluster = r_obj[1]
        centers = kmodel.clusterCenters()
        prediction_df = kmodel.transform(df_to_cluster)

        #set up rdd
        rdd = prediction_df.rdd.map(lambda (a,b):\
                                   (a, centers[b]))
        sse = rdd.map(lambda (a,b): compute_sse(a,b)).sum()
    return r_obj[0], df, scalerModels, sse, r_obj[1]
    #return r_obj[0], r_obj[1], scalerModels, sse

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers


#minmax
# def denormalize_cluster(cluster_centers, list_scaler_models):
#     r_centers = []
#     for center in cluster_centers:
#         r_center = []
#         for index in range(len(center)):
#             s_model = list_scaler_models[index]
#             r_center.append(s_model.denormalize(center[index]))
#         r_centers.append(r_center)
#     return r_centers


def denormalize_cluster(cluster_centers, list_scaler_models, columns):
    r_centers = []
    length = len(columns)
    for center in cluster_centers:
        r_center = []
        print center
        denormed_center = [0]*length
        
        #for each scaler models, get index in columns and denorm
        for k,v in list_scaler_models.iteritems():
            col_name = k
            model = v
            index = columns.index(col_name)
            #print index, 'masa2'
            denormed_center[index] = center[index]*model.std+model.mean
        r_centers.append(denormed_center)
        
#         for index in range(len(center)):
#             s_model = list_scaler_models[index]
#             #model = s_model.mmModel
#             denormed_val = center[index]*s_model.std+s_model.mean
#             r_center.append(denormed_val)
#         r_centers.append(r_center)
    return r_centers

def get_distribution_of_workouts(transformed_df):
    #count of predictions
    return transformed_df.groupBy('prediction').count().orderBy('prediction')

#transform (add prediction column) to the dataframe based on the kmeamodel input
def transform_df_with_kmeamodel(kmeanModel, df, cols_to_cluster, scale_dict={}):
    #need to vectorize and normalize the df
    df_to_transform, scalerModels = normalize_vectorize_df(big_sumstats, cols_to_cluster, scale_dict)
    df_to_transform = create_feature_column(df_to_transform, cols_to_cluster)
    return kmeanModel.transform(df_to_transform), scalerModels



#
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), r[7]]).toDF([\
            'prediction', 'userid', 'workoutid', 'label', 'max_elapsed_time'])

# total_df= fix_df(sumstats.join(prediction_df.select('workoutid','prediction'), \
#                                sumstats['workoutid']==prediction_df['workoutid'],'inner').\
#                             drop(prediction_df['workoutid']))

def denorm_df_with_standarscaler(dict_scalerModels, df):
    for k,v in dict_scalerModels.iteritems():
        #print '--denorm ', k, '--'
        col_name = k
        scaler = v
        mean = float(scaler.mean[0])
        std = float(scaler.std[0])
        denorm_udf = udf(lambda value: value*std + mean, FloatType())
        df = df.withColumn(col_name, denorm_udf(df[col_name]))
    return df
    
#     mean = float(scalerModel.mean[0])
#     std = float(scalerModel.std[0])
#     denorm_udf = udf(lambda value: value*std + mean, FloatType())
#     return df.withColumn()
    
    
    
    
#     def test(scaler, df):
#     dist_scaler = scaler
#     print df.describe().show(10,False)
#     #nd = df.withColumn('geo_distance', dist_scaler.s_udf(df['geo_distance']))
#     mean = float(dist_scaler.mean[0])
#     std = float(dist_scaler.std[0])
#     print mean,std
#     denorm_udf = udf(lambda value: value*std + mean, FloatType())
#     #denorm_udf = udf(lambda value: value*std, FloatType())

#     #denorm_udf = udf(lambda x: 'new_value', StringType())

#     r_df = df.withColumn('geo_distance', denorm_udf(df['geo_distance']))



time: 972 ms


In [64]:
#magic
unvector = udf(lambda value: value[0].item(), FloatType())

time: 6.71 ms


In [9]:
from pyspark.sql import DataFrameReader

url = 'postgresql://34.199.246.37:5432/endomondo_sample'
#url = 'postgresql://172.31.29.1:5432/endomondo'

properties = {'user':'endomondo', 'password': 'End0m0ndo'}

df_runs = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run', properties=properties)

df_users = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run_by_workout', properties=properties)

df_runs

DataFrame[altitude: decimal(20,10), heart_rate: decimal(10,5), latitude: decimal(20,10), longitude: decimal(20,10), speed: decimal(20,10), workoutid: int, time: int, altitude_first: decimal(10,5), altitude_second: decimal(10,5), elapsed_time: int, speed_first: decimal(10,5), geo_distance: decimal(20,10), elapsed_distance: decimal(15,10), heart_rate_ma_25: decimal(8,5), speed_ma_100: decimal(8,5), speed_ma_50: decimal(8,5), speed_by_geo: decimal(20,10)]

time: 1.57 s


In [10]:
df_users.count()

25000

time: 1.78 s


In [65]:
sub_df = df_runs.limit(1000)
df_users = df_users.withColumn('diff_altitude', \
                    df_users['altitude_max'] \
                    -df_users['altitude_min'])
sumstats = df_users.repartition(30)
#sumstats = df_users.limit(500).repartition(30)

time: 13.6 ms


In [12]:
sumstats.select('diff_altitude').describe().show()



+-------+-----------------+
|summary|    diff_altitude|
+-------+-----------------+
|  count|            19671|
|   mean|91.14937387423110|
| stddev|444.8822654794946|
|    min|            0E-10|
|    max| 13103.6000000000|
+-------+-----------------+

time: 740 ms


In [41]:
route_cols = ['diff_altitude', 'geo_distance']
#route_cols = ['diff_altitude']


time: 1.05 ms


In [66]:
scale_dict = {}

r_obj = cluster_df(sumstats, route_cols, 5, init_var=True,\
                  scale_dict=scale_dict, bisectingKMean=True,\
                  debug=True)
print 'done'

cluster start
vectorize done in: 0:00:00.051074
------- normalize  diff_altitude -------
0:00:00.368397
0:00:00.051474
------- normalize  geo_distance -------
0:00:02.383062
0:00:00.053614
clustering in  0:00:38.390130
done
time: 41.3 s


In [87]:
print route_df.select(route_cols).describe().show()


+-------+--------------------+--------------------+
|summary|       diff_altitude|        geo_distance|
+-------+--------------------+--------------------+
|  count|               24876|               24876|
|   mean|-0.05221769265578752|-0.00732095255894...|
| stddev| 0.22178324766198665| 0.17670662451058466|
|    min|         -0.18093379|         -0.27459386|
|    max|            2.992725|           2.9708807|
+-------+--------------------+--------------------+

None
time: 5.36 s


In [89]:
print df_to_denorm.select(route_cols).describe().show()


+-------+--------------------+--------------------+
|summary|       diff_altitude|        geo_distance|
+-------+--------------------+--------------------+
|  count|               24876|               24876|
|   mean|-0.05221769265578752|-0.00732095255894...|
| stddev| 0.22178324766198665| 0.17670662451058466|
|    min|         -0.18093379|         -0.27459386|
|    max|            2.992725|           2.9708807|
+-------+--------------------+--------------------+

None
time: 5.7 s


In [67]:
route_kmodel = r_obj[0]
route_df = r_obj[4]
route_scalerModels = r_obj[2]
df_to_denorm = r_obj[1]
#route_df.count()

time: 40.7 ms


In [68]:
for k,v in route_scalerModels.iteritems():
    print k,v

geo_distance StandardScaler_4853be56338659051f1c
diff_altitude StandardScaler_466dbcd659afa3ccd900
time: 1.51 ms


In [69]:
route_df.count()

24876

time: 3.85 s


In [46]:
###check if routes make sense
#route_df.select(route_cols).toPandas().describe()

time: 685 µs


In [90]:
route_prediction_df = route_kmodel.transform(route_df)
#print route_prediction_df.count()
route_prediction_cols = ['route_prediction','route_features']
distinct_clusters = route_prediction_df.select('prediction').distinct().collect()
route_prediction_df = route_prediction_df.withColumnRenamed('prediction', 'route_prediction')
route_prediction_df = route_prediction_df.withColumnRenamed('features', 'route_features')

route_prediction_df = denorm_df_with_standarscaler(route_scalerModels, route_prediction_df)


time: 5.16 s


In [96]:
route_prediction_df.select(route_cols).describe().show(10,False)

+-------+---------------------+--------------------+
|summary|diff_altitude        |geo_distance        |
+-------+---------------------+--------------------+
|count  |24876                |24876               |
|mean   |51.02150974717797    |0.13627124255003475 |
|stddev |87.9122041826895     |0.09009529141855512 |
|min    |1.8724292658589548E-6|4.432102684859274E-9|
|max    |1258.0               |1.654731273651123   |
+-------+---------------------+--------------------+

time: 4.03 s


In [91]:
route_prediction_df.groupBy('route_prediction').count().show()
route_prediction_df = fix_df(route_prediction_df)

+----------------+-----+
|route_prediction|count|
+----------------+-----+
|               1|  142|
|               3| 4575|
|               4| 2711|
|               2|  630|
|               0|16818|
+----------------+-----+

time: 11.2 s


In [92]:
route_kmodel.clusterCenters()

[array([-0.12519179, -0.07004625]),
 array([ 1.95885868,  0.19198119]),
 array([ 0.66136054,  0.09302346]),
 array([ 0.07946549,  0.00195137]),
 array([-0.07879936,  0.33516802])]

time: 6.84 ms


In [82]:
denormed_clusters = denormalize_cluster(route_kmodel.clusterCenters(),  route_scalerModels, route_cols)
#save cluster centers for route
#unvector = udf(lambda value: value[0].item(), FloatType())
route_viz_cols = route_cols
for i in range(len(denormed_clusters)):
    for j in range(len(route_viz_cols)):
        denormed_clusters[i][j] = float(denormed_clusters[i][j][0])
print denormed_clusters

[-0.12519179 -0.07004625]
[ 1.95885868  0.19198119]
[ 0.66136054  0.09302346]
[ 0.07946549  0.00195137]
[-0.07879936  0.33516802]
[[22.095457609561755, 0.1042902478691724], [848.1879279535751, 0.2378870411397166], [333.8753130262295, 0.18743264714643124], [103.21913428407004, 0.140998811351551], [40.48485885947855, 0.3108919976252643]]
time: 58.1 ms


In [22]:
sc.parallelize(denormed_clusters).toDF(route_viz_cols).toPandas().to_csv('route_clusters.csv')

time: 419 ms


denormalize_cluster(route_kmodel.clusterCenters(),  route_scalerModels)

In [23]:
distribution_df = get_distribution_of_workouts(route_kmodel.transform(route_df))
distribution_df.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0| 7087|
|         1|15051|
|         2| 1730|
|         3|  223|
|         4|  785|
+----------+-----+

time: 5.37 s


In [68]:
#######################end route checker

time: 782 µs


In [83]:
#get cluster numbers
cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]
cluster_numbers

[1, 3, 4, 2, 0]

time: 4.02 ms


In [93]:
#create list of dataframes for each route_prediction value
list_route_clusters = [route_prediction_df.where(route_prediction_df['route_prediction']==i) \
                      for i in cluster_numbers]

time: 20.8 ms


In [72]:
list_route_clusters[0].schema.names

['gender',
 'workoutid',
 'userid',
 'start_time',
 'start_altitude',
 'start_latitude',
 'start_longitude',
 'series_length',
 'series_time_delta',
 'series_time_delta_average',
 'timezone',
 'id',
 'altitude_max',
 'altitude_min',
 'calories',
 'distance',
 'duration',
 'heart_rate_avg',
 'heart_rate_max',
 'hydration',
 'speed_avg',
 'speed_max',
 'humidity',
 'temperature',
 'wind_speed',
 'elapsed_time',
 'time_since_last_workout',
 'diff_altitude',
 'geo_distance',
 'route_features',
 'route_prediction']

time: 3.88 ms


In [29]:
list_route_clusters[0].select(['diff_altitude','geo_distance','speed_avg']).describe().show(10)

KeyboardInterrupt: 

time: 5.14 s


In [99]:
#got list of dataframes by route cluster. for each of them, create new clusters 
#add a new column 'performance_prediction' for each cluster

performance_cols_to_cluster = ['heart_rate_avg', 'speed_avg', 'elapsed_time']

dict_cluster_to_objs = {}

n_list_route_clusters = []

perf_centers_df_denormed = []

for i in range(len(list_route_clusters)):
    print i, 
#     #cluster
#     c_df = list_route_clusters[i]
#     r_obj = cluster_df(c_df, performance_cols_to_cluster, 5, init_var=True, \
#                       scale_dict = scale_dict, bisectingKMean=True,\
#                       debug=False)
#     dict_cluster_to_objs[i] = r_obj

#     #transform the dataset
#     p_kmodel = r_obj[0]
#     p_df_to_transform = r_obj[1]
#     p_scalerModels = r_obj[2]
    
#     p_kmodel.transform(p_df_to_transform)
    c_df = list_route_clusters[i].repartition(20)
    #print c_df.select(route_cols + performance_cols_to_cluster).describe().show(10,False)
    #cluster within the route cluster dataframe
    a = datetime.now()

    r_obj = cluster_df(c_df, performance_cols_to_cluster, 3, init_var=True, \
                      scale_dict = scale_dict, bisectingKMean=True,\
                      debug=False)
    
    b = datetime.now()
    print 'cluster done', b-a
    dict_cluster_to_objs[i] = r_obj

    #transform the dataset
    p_kmodel = r_obj[0]
    p_df_to_transform = r_obj[4]
    p_scalerModels = r_obj[2]
    for k,v in p_scalerModels.iteritems():
        print k,v
    print 'transforming'
    temp_df = p_kmodel.transform(p_df_to_transform)
    temp_df = temp_df.withColumnRenamed('features', 'perf_features')
    temp_df = temp_df.withColumnRenamed('prediction', 'perf_prediction')
    #print c_df.schema
    
    denormed_center = denormalize_cluster(p_kmodel.clusterCenters(),  p_scalerModels, performance_cols_to_cluster)
    for i in range(len(denormed_center)):
        for j in range(len(performance_cols_to_cluster)):
            denormed_center[i][j] = float(denormed_center[i][j][0])
    
    perf_centers_df_denormed.append(\
                                    sc.parallelize(denormed_center)\
                                    .toDF(performance_cols_to_cluster))
    
    
    print '\n'
    #transform
    c_df = c_df.drop('perf_features')
    c_df = c_df.drop('perf_prediction')
    a = datetime.now()

    #list_route_clusters[i] = c_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
    #                                  temp_df['workoutid']==c_df['workoutid'], 'inner')\
    #                                    .drop(temp_df['workoutid'])
#     n_list_route_clusters.append(c_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
#                                       temp_df['workoutid']==c_df['workoutid'], 'inner')\
#                                         .drop(temp_df['workoutid']))
    
    #print r_obj[1].select(route_cols).describe().show(10,False)
    denormed_df = denorm_df_with_standarscaler(p_scalerModels, r_obj[1])
    
    #denorm route info
    #denormed_df = denorm_df_with_standarscaler(route_scalerModels, denormed_df)

    
    n_list_route_clusters.append(denormed_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
                                      temp_df['workoutid']==denormed_df['workoutid'], 'inner')\
                                        .drop(temp_df['workoutid']))
    
    b = datetime.now()
    print 'joined', b-a
    
    #print denormed_df.select(route_cols + performance_cols_to_cluster).describe().show(10,False)
    #break
    #list_route_clusters[i].show()
    #break
    


0 cluster start
vectorize done in: 0:00:00.077889
cluster done 0:00:24.236032
elapsed_time StandardScaler_47c5b55aee435a1d6b88
heart_rate_avg StandardScaler_415bb8b48b5acd79c8a9
speed_avg StandardScaler_4d04b35cb69570330164
transforming
[-0.03976819 -0.51834702  0.02008771]
[-6.52477884 -1.25541839  0.68115728]
[ 0.43542948  1.16146665 -0.56828775]


joined 0:00:00.129802
1 cluster start
vectorize done in: 0:00:00.077336
cluster done 0:00:30.264793
elapsed_time StandardScaler_47228aea13008b0d9267
heart_rate_avg StandardScaler_4562a841079738c4f600
speed_avg StandardScaler_411789b26ab1553c9d6c
transforming
[ 0.36744106  0.55491902 -0.02846549]
[ 0.1783769  -0.82193714 -0.00316717]
[-3.10485956  0.08690412 -0.01893489]


joined 0:00:00.130059
2 cluster start
vectorize done in: 0:00:00.079202
cluster done 0:00:30.192969
elapsed_time StandardScaler_427f8541b3a567010616
heart_rate_avg StandardScaler_480b8574051e8e3fddc2
speed_avg StandardScaler_450fbcce2e79637f141a
transforming
[ 0.44779982 

In [100]:
sumstats.select(['diff_altitude']).describe().show()

+-------+-----------------+
|summary|    diff_altitude|
+-------+-----------------+
|  count|            19671|
|   mean|91.14937387423110|
| stddev|444.8822654794946|
|    min|            0E-10|
|    max| 13103.6000000000|
+-------+-----------------+

time: 149 ms


In [101]:
new_features=[ 'workoutid',\
 'route_prediction',\
 'perf_prediction', 'userid']

time: 1.07 ms


In [102]:
list_total_dfs = [i.select(new_features + route_cols + performance_cols_to_cluster) for i in n_list_route_clusters]

time: 133 ms


In [103]:
list_total_dfs[0].schema.names

['workoutid',
 'route_prediction',
 'perf_prediction',
 'userid',
 'diff_altitude',
 'geo_distance',
 'heart_rate_avg',
 'speed_avg',
 'elapsed_time']

time: 4.71 ms


In [ ]:
list_total_dfs[0].select(cols_viz).describe().show()

In [30]:
dd = list_total_dfs[0]

time: 872 µs


In [104]:
#use total_df to find avg_spd/avg_dist for each user in each cluster
# from the predictions, group by userid/clusterid and calculate avg speed and avg distance for each user

w = Window.partitionBy('userid', 'route_prediction')

new_speed = 'user_avg_speed'
new_dist = 'user_avg_dist'

for i in range(len(list_total_dfs)):
    df = list_total_dfs[i].repartition(30)
    df_speed = df.select('userid', 'route_prediction',\
                          avg('speed_avg').over(w).alias(new_speed)).distinct()
    df_dist = df.select('userid', 'route_prediction',\
                          avg('geo_distance').over(w).alias(new_dist)).distinct()
    total = df_speed.join(df_dist,(df_speed.route_prediction==df_dist.route_prediction)\
                           & (df_speed.userid==df_dist.userid),\
                     'inner').drop(df_speed.route_prediction).drop(df_speed.userid)
    
    
    list_total_dfs[i] = df.join(total, total.userid==df.userid, \
                                'inner').drop(total.route_prediction)\
                                .drop(total.userid)
        



time: 2.35 s


In [32]:
list_total_dfs[0].count()

15012

time: 20.5 s


In [38]:

cols_viz = ['diff_altitude', 'geo_distance', 'heart_rate_avg', 'speed_avg', 'user_avg_dist', 'user_avg_speed']

#list_total_dfs[1].select(cols_viz).describe().show(truncate=10)

time: 1.34 ms


In [30]:
###check avg
#list_total_dfs[2].where(col('userid')==6601602).show()

+---------+----------------+---------------+------+-------------+------------+--------------+---------+------------+--------------+-------------+
|workoutid|route_prediction|perf_prediction|userid|diff_altitude|geo_distance|heart_rate_avg|speed_avg|elapsed_time|user_avg_speed|user_avg_dist|
+---------+----------------+---------------+------+-------------+------------+--------------+---------+------------+--------------+-------------+
+---------+----------------+---------------+------+-------------+------------+--------------+---------+------------+--------------+-------------+

time: 25.9 s


In [105]:
#make one df from list_total_df
new_concat_df = unionAll(*list_total_dfs)

time: 443 ms


In [106]:
new_concat_df.count()

24654

time: 1min 7s


In [35]:
#dd = new_concat_df.select(cols_viz)

time: 166 ms


In [107]:
#dd.describe().show(10, True)

time: 538 µs


In [107]:
new_concat_df.describe().show(truncate=10)

+-------+----------+----------------+---------------+----------+-------------+------------+--------------+----------+------------+--------------+-------------+
|summary| workoutid|route_prediction|perf_prediction|    userid|diff_altitude|geo_distance|heart_rate_avg| speed_avg|elapsed_time|user_avg_speed|user_avg_dist|
+-------+----------+----------------+---------------+----------+-------------+------------+--------------+----------+------------+--------------+-------------+
|  count|     24654|           24654|          24654|     24654|        24654|       24654|         24654|     24654|       24654|         24654|        24654|
|   mean|3.63313...|      1.03139...|     0.68504...|4546589...|   50.6747...|  0.13424...|    131.244...|10.7855...|  4380.01...|    10.7855...|   0.13424...|
| stddev|1.62107...|      1.54710...|     0.82563...|3802550...|   86.7839...|  0.08359...|    48.1253...|2.32509...|  9191.96...|    1.77780...|   0.06968...|
|    min|     32946|               0|   

In [108]:
!ls

autotime.py				   mmscaler_wrapper.py
autotime.pyc				   mmscaler_wrapper.pyc
check_free_space.sh			   mycert.pem
check_memory.sh				   mykey.key
cluster_centers.csv			   new_run2.csv
cluster_centers_gt10.csv		   notebooks
dave2.csv				   pypy
dave_.csv				   README.md
derby.log				   Regression 
f_cluster.csv				   Regression 1st Iteration
f_clusters4.csv				   Regression_2nd_Iteration
f_clusters5.csv				   Regression_3rd_Iteration
f_clusters6.csv				   route_clusters.csv
import_influx.py			   route_perf_clusters.csv
Jason-Loading-Visualizing-Endomondo.ipynb  run_data_with_hr_spd.csv
Larger Cluster Regression.ipynb		   run_with_hr_users.csv
masa-cluster series.ipynb		   save-package-versions
masa-cluster summary.ipynb		   sc_test.ipynb
masa-newclustering-Copy1.ipynb		   sql_scripts
masa-newclustering.ipynb		   test.csv
metastore_db.old			   visualize_cluster.ipynb
mmscaler_wrapper.ipynb			   visualize_clusters.html
time: 116 ms


In [109]:
new_concat_df.toPandas().to_csv('dave_.csv')

time: 1min 11s


In [195]:
!pwd

/Users/momori/dse/maomori/thesis/git/fitness_capstone
time: 125 ms


In [36]:
new_concat_df.groupby(['route_prediction','perf_prediction'])\
    .count().show()

+----------------+---------------+-----+
|route_prediction|perf_prediction|count|
+----------------+---------------+-----+
|               1|              1| 8764|
|               3|              1|   77|
|               3|              0|  125|
|               2|              0| 1582|
|               0|              0| 3100|
|               0|              1|  788|
|               1|              2| 1688|
|               2|              2|   62|
|               2|              1|   70|
|               4|              2|  617|
|               4|              1|   68|
|               3|              2|   10|
|               1|              0| 4560|
|               0|              2| 3106|
|               4|              0|   77|
+----------------+---------------+-----+

time: 1min 7s


In [111]:
new_concat_df.select('speed_avg').describe().show(10,False)

+-------+------------------+
|summary|speed_avg         |
+-------+------------------+
|count  |24654             |
|mean   |10.785569427380436|
|stddev |2.3250990751402534|
|min    |-3.2287147        |
|max    |23.394915         |
+-------+------------------+

time: 1min 28s


In [26]:
prediction_columns = ['route_prediction', 'perf_prediction']
#frames = [df.toPandas() for df in n_list_route_clusters]

time: 809 µs


In [27]:
#new_concat_df = sqlContext.createDataFrame(pd.concat(frames))

time: 543 µs


In [28]:
index = 0
for i in perf_centers_df_denormed:
    name='perf_clusters'+str(index)+'.csv'
    print name
    i.toPandas().to_csv(name)
    print i.show()
    index+=1

perf_clusters0.csv
+-----------------+------------------+-----------------+
|   heart_rate_avg|         speed_avg|     elapsed_time|
+-----------------+------------------+-----------------+
|134.4350000154713|11.513321991868393|6562.666684570312|
+-----------------+------------------+-----------------+

None
perf_clusters1.csv
+-----------------+-----------------+-----------------+
|   heart_rate_avg|        speed_avg|     elapsed_time|
+-----------------+-----------------+-----------------+
|162.6259991907229|15.18740863941769|6259.999990805761|
+-----------------+-----------------+-----------------+

None
time: 62.4 ms


In [31]:
new_concat_df = unionAll(*n_list_route_clusters)

time: 25.8 ms


In [32]:
new_concat_df.groupBy(['route_prediction','perf_prediction']).count()\
.orderBy(['route_prediction', 'perf_prediction']).show()

+----------------+---------------+-----+
|route_prediction|perf_prediction|count|
+----------------+---------------+-----+
|               1|              0|    3|
|               3|              0|    1|
+----------------+---------------+-----+

time: 3.88 s


In [75]:
#remove small clusters, might be outliers
new_concat_df = new_concat_df.where(
    ((new_concat_df['route_prediction']!= 4) | (new_concat_df['perf_prediction']!= 0))
)

time: 65.5 ms


In [33]:
select_cols = prediction_columns+route_cols+['userid','workoutid']+performance_cols_to_cluster
print select_cols

['route_prediction', 'perf_prediction', 'diff_altitude', 'geo_distance', 'userid', 'workoutid', 'heart_rate_avg', 'speed_avg', 'elapsed_time']
time: 1.16 ms


In [34]:
sum_cols = route_cols+performance_cols_to_cluster+['workoutid']


time: 720 µs


In [35]:
new_concat_df.select(select_cols).show()


+----------------+---------------+------------------+------------------+--------+---------+--------------------+--------------------+------------+
|route_prediction|perf_prediction|     diff_altitude|      geo_distance|  userid|workoutid|      heart_rate_avg|           speed_avg|elapsed_time|
+----------------+---------------+------------------+------------------+--------+---------+--------------------+--------------------+------------+
|               1|              0|0.2209971696138382|0.7775065898895264|  431375|608308965|141.0000000000000...|12.01837677640000...|        6493|
|               1|              0|0.2412315011024475| 0.919242799282074| 9286593|367180538|122.3050000000000...|11.40694893660000...|        6317|
|               1|              0|0.2264968752861023|0.7706879377365112| 6828710|475598541|140.0000000000000...|11.11464029460000...|        6878|
|               3|              0|0.6398912072181702|1.7554370164871216|13276532|376398864|162.6260000000000...|15.187

In [36]:
f_cluster_df = new_concat_df.select(select_cols)
f_cluster_df = f_cluster_df.withColumn('diff_altitude', unvector(f_cluster_df['diff_altitude']))
f_cluster_df = f_cluster_df.withColumn('geo_distance', unvector(f_cluster_df['geo_distance']))



time: 75.4 ms


In [37]:
new_df = fix_df(new_concat_df.select(select_cols))

time: 2.3 s


In [85]:
viz_selection_route = ['geo_distance', 'diff_altitude']

new_df.select(viz_selection_route).toPandas().to_csv('f_clusters6.csv')

time: 9.44 s


In [86]:
new_df.select(viz_selection_route).show()

+--------------------+--------------------+
|        geo_distance|       diff_altitude|
+--------------------+--------------------+
|-0.23650439083576202|-0.18164116144180298|
| -0.2968972623348236| 0.23747123777866364|
| -0.2613212466239929|-0.11914194375276566|
| -0.2013634741306305| 0.12644322216510773|
|-0.24943852424621582|-0.10811267048120499|
|-0.26658928394317627|-0.18164116144180298|
|-0.37416771054267883|  0.3808518052101135|
| -0.2730995714664459|-0.11473023146390915|
|-0.28780242800712585|-0.09703560173511505|
|-0.23023466765880585|-0.13568584620952606|
| -0.1474791318178177| -0.1125243753194809|
| -0.1570734828710556|-0.03237832337617874|
|-0.04410969838500023| 0.47202712297439575|
|-0.14297319948673248|-0.10664209723472595|
|-0.15701673924922943|-0.18164116144180298|
| -0.1453741192817688|-0.13458292186260223|
|-0.28630220890045166| 0.12864907085895538|
|-0.15556730329990387|-0.02943718247115612|
|-0.22201518714427948|-0.13642114400863647|
|-0.24090249836444855|  0.351440

In [83]:
new_df.write.csv('f_clusters7.csv')

time: 19 s


In [94]:
f_cluster_df.toPandas().to_csv('f_clusters5.csv')

Py4JJavaError: An error occurred while calling o2274.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 23 in stage 2376.0 failed 4 times, most recent failure: Lost task 23.3 in stage 2376.0 (TID 145990, 172.31.42.143, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/opt/spark/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 106, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "<string>", line 1, in <lambda>
  File "/opt/spark/spark-2.1.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 70, in <lambda>
    return lambda *a: f(*a)
  File "<ipython-input-54-6

time: 1.93 s


In [93]:
f_cluster_df.write.csv('f_clusters5.csv')

Py4JJavaError: An error occurred while calling o2275.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:147)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apach

time: 2.33 s


In [98]:
f_cluster_df.toPandas().describe()

,route_prediction,perf_prediction,diff_altitude,geo_distance,userid,workoutid,elapsed_time
count,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1000.000000
mean,1.606000,1.231000,0.005290,0.035301,4.742135e+06,4.056705e+08,3622.448000
std,1.102715,0.745785,0.032204,0.040235,3.977771e+06,1.524253e+08,3754.103925
min,0.000000,0.000000,0.000000,0.000000,6.900000e+01,9.894440e+05,506.000000
25%,1.000000,1.000000,0.000000,0.019071,1.607905e+06,3.126698e+08,2134.500000
50%,2.000000,1.000000,0.002490,0.031582,3.575734e+06,4.162176e+08,3093.000000
75%,3.000000,2.000000,0.005310,0.044650,7.482039e+06,5.252203e+08,4217.500000
max,3.000000,2.000000,1.000000,1.000000,1.548142e+07,6.551612e+08,84174.000000


time: 282 ms


In [42]:
f_cluster_df.write.csv('f_cluster2.csv')

time: 3.56 s


In [45]:
f_cluster_df.write.parquet('f_cluster3.csv')

time: 3.41 s


In [49]:
schema=f_cluster_df.schema.names

time: 1.1 ms


In [55]:
sqlContext.read.parquet('f_cluster3.csv')

AnalysisException: u'Unable to infer schema for Parquet. It must be specified manually.;'

time: 75 ms


In [58]:
f_cluster_df.toPandas().to_csv('f_clusters4.csv')

time: 3.56 s


In [59]:
pd.read_csv('f_clusters4.csv')

,Unnamed: 0,route_prediction,perf_prediction,diff_altitude,geo_distance,userid,workoutid,heart_rate_avg,speed_avg,elapsed_time
0,0,1,2,0.000000,0.050324,43462,22952825,179.000,9.931427,0
1,1,1,2,0.003118,0.049722,1038205,29839134,176.577,14.940946,1016534
2,2,1,2,0.004915,0.049018,114073,33675191,168.000,14.060335,25389273
3,3,1,2,0.002494,0.054654,3276737,56085342,162.796,10.481048,14255297
4,4,1,2,0.011665,0.047487,324779,60963533,155.061,9.609221,15717111
5,5,1,2,0.000000,0.050781,860533,97645888,164.549,11.894138,2190253
6,6,1,2,0.004655,0.054519,2255761,100089344,151.000,10.962511,18987867
7,7,1,1,0.018911,0.061267,81753,156154748,159.129,10.201762,112583114
8,8,1,2,0.000000,0.049694,5325166,172545666,147.000,8.590226,22200235
9,9,1,1,0.002854,0.055214,983950,181978674,146.923,13.294408,63173301


time: 107 ms


In [6]:
df_fro

IllegalArgumentException: u'Can not create a Path from an empty string'

time: 505 ms


In [56]:
print c

None
time: 1.19 ms


In [49]:
unv_df = route_df.select(['diff_altitude', 'geo_distance'])
unv_df = unv_df.withColumn('diff_altitude', unvector(route_df['diff_altitude']))
unv_df = unv_df.withColumn('geo_distance', unvector(route_df['geo_distance']))

unv_df.show()

+-------------+------------+
|diff_altitude|geo_distance|
+-------------+------------+
|  0.014819317| 4.512356E-6|
| 0.0105275875|2.9843734E-6|
|  0.004059953| 2.551286E-6|
|  0.005942405|1.9085373E-6|
| 0.0028244788|3.0166398E-6|
| 0.0020299766|1.8026603E-6|
| 0.0014347194|2.0295515E-6|
| 0.0027931256|1.8971801E-6|
| 0.0025310297|  2.81416E-6|
| 0.0022742606|1.3907281E-6|
| 6.2578224E-4| 5.473361E-6|
|  0.002129186|4.9994883E-6|
| 0.0039683753| 2.141207E-6|
| 0.0025947068|5.0974104E-6|
| 0.0025676913|3.7508262E-6|
| 0.0026786532|1.2864327E-6|
| 0.0022436583| 2.459735E-6|
| 0.0035104856|3.0550423E-6|
| 0.0017247169|2.2072018E-6|
|  0.004075216|1.9128806E-6|
+-------------+------------+
only showing top 20 rows

time: 887 ms


In [55]:
p_df = unv_df.select(['diff_altitude', 'geo_distance']).toPandas()

time: 2.7 s


In [56]:
p_df.describe()

,diff_altitude,geo_distance
count,25000.000000,25000.000000
mean,0.005473,0.000003
std,0.030250,0.000014
min,0.000000,0.000000
25%,0.000626,0.000002
50%,0.002366,0.000002
75%,0.004579,0.000003
max,1.000000,0.001570


time: 15.8 ms


In [57]:
p_df.head(10)

,diff_altitude,geo_distance
0,0.014819,0.000005
1,0.010528,0.000003
2,0.004060,0.000003
3,0.005942,0.000002
4,0.002824,0.000003
5,0.002030,0.000002
6,0.001435,0.000002
7,0.002793,0.000002
8,0.002531,0.000003
9,0.002274,0.000001


time: 8.06 ms


In [65]:
route_cols = ['diff_altitude', 'geo_distance']
df = vectorize_columns(sumstats, route_cols)
df.schema.names

vectorize done in 0:00:00.050535


['gender',
 'workoutid',
 'userid',
 'start_time',
 'start_altitude',
 'start_latitude',
 'start_longitude',
 'series_length',
 'series_time_delta',
 'series_time_delta_average',
 'timezone',
 'id',
 'altitude_max',
 'altitude_min',
 'calories',
 'distance',
 'duration',
 'heart_rate_avg',
 'heart_rate_max',
 'hydration',
 'speed_avg',
 'speed_max',
 'humidity',
 'temperature',
 'wind_speed',
 'elapsed_time',
 'diff_altitude',
 'geo_distance']

time: 56.1 ms


In [64]:
normalize_df4(df, route_cols, debug=True)

------- normalize  diff_altitude -------


ValueError: Can not reduce() empty RDD

time: 2.21 s


In [58]:
def normalize_df4(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict

time: 13.6 ms


In [ ]:
f_cluster_df = f_cluster_df.withColumn('diff_altitude', unvector(f_cluster_df['diff_altitude']))
f_cluster_df = f_cluster_df.withColumn('geo_distance', unvector(f_cluster_df['geo_distance']))